<a href="https://colab.research.google.com/github/dkurbatovv/Python/blob/main/Real_and_Fake_Pokemon_Cards.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
!mkdir -p ~/.kaggle/

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d ongshujian/real-and-fake-pokemon-cards

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
import glob as gb

import os
import cv2
from sklearn import preprocessing
from pathlib import Path

In [ ]:
images = []

dataset = "/content/image/"

for folder in os.listdir(dataset):
  files = gb.glob(pathname=str(dataset + folder + "/*.JPG"))
  for file in files:
    image = cv2.imread(file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    image = cv2.resize(image, (256,256))

    images.append(image)

images = np.array(images)

 

In [ ]:
images

In [ ]:
labels = pd.read_csv('train_labels.csv')

In [ ]:
labels['id'] = labels['id'].astype('str')
labels = labels.sort_values(by=['id'], ignore_index = True)

In [ ]:
labels.drop('id', axis = 1)

In [ ]:
labels['categotical'] = labels['label'].apply(lambda x : 'fake' if x == 0 else 'real')

In [ ]:
labels = labels.drop(['id', 'label'], axis = 1)

In [ ]:
unique, counts = np.unique(labels, return_counts = True)

In [ ]:
counts

In [ ]:
import seaborn as sns
sns.barplot(x = unique, y = counts)


In [ ]:
labels = np.array(pd.get_dummies(labels))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_Test, y_train, y_Test = train_test_split(images, labels, test_size = 0.3, random_state = 100)


In [ ]:
plt.imshow(X_train[100])
plt.title('Its a '+ str(y_train[100]))

In [ ]:
plt.imshow(X_train[15])
plt.title('Its a '+ str(y_train[15]))

In [ ]:
X_train = X_train / 255.0
X_Test = X_Test / 255.0

In [ ]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (3,3), input_shape = (256,256, 3), activation = 'relu', padding = 'same'))
model.add(Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.2))

model.add(Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu', padding = 'same'))
model.add(Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.2))

model.add(Conv2D(filters = 128, kernel_size = (3,3), activation = 'relu', padding = 'same'))
model.add(Conv2D(filters = 128, kernel_size = (3,3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.2))


model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2, activation = 'softmax')) #тк у нас либо фейк либо не фейк берем функцию сигмоиды, либо 1 либо 0

In [ ]:
opt = Adam(lr = 0.01)

In [ ]:
model.summary()

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystopper = EarlyStopping(monitor = 'val_loss', patience=2, verbose=1, restore_best_weights=True)
reducel = ReduceLROnPlateau(monitor = 'val_loss', patience=1, verbose=1, factor=0.1)

In [ ]:
model.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs = 50, validation_split = 0.25)